Travail réalisé par Marc le Chevoir, Claire Espérou et Adrien Schaffner, encadrés par Philippe Besse.

# Etude du jeu de données "Maths" ou "Portugais" avec R

In [ ]:
library(vcd)
library(FactoMineR)
library(corrplot)
library(boot)
library(ROCR) 
library(doParallel)

In [ ]:
cls = makeCluster(4)
registerDoParallel(cls)

# 1 - Importation des données

Choisir le jeu de données : soit l'évaluation en Maths, soit en Portugais

In [ ]:
Data=read.table("./student/student-mat.csv",sep=";",header=TRUE)
#Data=read.table("./student/student-por.csv",sep=";",header=TRUE)

In [ ]:
dim(Data)
Data

Suppression des individus ayant 0 en note finale : indidivus non utilisables.

In [ ]:
Data = Data[Data$G3!=0,]
dim(Data)

### Listes des variables, regroupées par type :

In [ ]:
variables_descriptives = c("school","sex","address","famsize","Pstatus","Mjob","Fjob","reason","guardian","schoolsup",
                   "famsup","paid","activities","nursery","higher", "internet","romantic")
variables_numeriques = c("age","absences","G1","G2","G3","Medu","Fedu","traveltime", "studytime","failures","famrel",
                         "freetime","goout","Dalc","Walc","health")

Attention aux variables qualitatives ordinales (Medu, Fedu ...) considérées ici comme quantitatives : ce choix peut être remis en question. Il faudra prendre garde lors de la création des modèles de prévision si on considère que ce sont des variables qualitatives ou quantitatives.

# 2 - Transformation des données

### Transformation des variables descriptives (quantitatives) en variables qualitatives :

In [ ]:
dataq<-Data
dataq[,"age"]<-cut(Data$age,
                   breaks=quantile(Data[,"age"], probs = seq(0, 1, 1/4)),
                   labels = c("A0","A1","A2","A3"),include.lowest = TRUE)
dataq[,"absences"]<-cut(Data$absences,
                        breaks=quantile(Data[,"absences"], probs = seq(0, 1, 1/2)),
                        labels = c("Abs0","Abs1"),include.lowest = TRUE)
dataq[,"G1"]<-cut(Data$G1,
                  breaks=quantile(Data[,"G1"], probs = seq(0, 1, 1/4)),
                  labels = c("G1_0","G1_1","G1_2","G1_3"),include.lowest = TRUE)
dataq[,"G2"]<-cut(Data$G2,
                  breaks=quantile(Data[,"G2"], probs = seq(0, 1, 1/4)),
                  labels = c("G2_0","G2_1","G2_2","G2_3"),include.lowest = TRUE)
dataq[,"G3"]<-cut(Data$G3,
                  breaks=quantile(Data[,"G3"], probs = seq(0, 1, 1/4)),
                  labels = c("G3_0","G3_1","G3_2","G3_3"),include.lowest = TRUE)

### Renommage des modalités :

In [ ]:
dataq[,"Medu"]<-factor(Data[,"Medu"],
                     levels=c("0","1","2","3","4"),
                     labels=c("Medu_0","Medu_1","Medu_2","Medu_3","Medu_4"))
dataq[,"Fedu"]<-factor(Data[,"Fedu"],
                     levels=c("0","1","2","3","4"),
                     labels=c("Fedu_0","Fedu_1","Fedu_2","Fedu_3","Fedu_4"))
dataq[,"traveltime"]<-factor(Data[,"traveltime"],
                     levels=c("1","2","3","4"),
                     labels=c("travel_1","travel_2","travel_3","travel_4"))
dataq[,"studytime"]<-factor(Data[,"studytime"],
                     levels=c("1","2","3","4"),
                     labels=c("study_1","study_2","study_3","study_4"))
dataq[,"failures"]<-factor(Data[,"failures"],
                     levels=c("1","2","3","4"),
                     labels=c("failures_1","failures_2","failures_3","failures_4"))
dataq[,"famrel"]<-factor(Data[,"famrel"],
                     levels=c("1","2","3","4","5"),
                     labels=c("famrel_1","famrel_2","famrel_3","famrel_4","famrel_5"))
dataq[,"freetime"]<-factor(Data[,"freetime"],
                     levels=c("1","2","3","4","5"),
                     labels=c("freetime_1","freetime_2","freetime_3","freetime_4","freetime_5"))
dataq[,"goout"]<-factor(Data[,"goout"],
                     levels=c("1","2","3","4","5"),
                     labels=c("goout_1","goout_2","goout_3","goout_4","goout_5"))
dataq[,"Dalc"]<-factor(Data[,"Dalc"],
                     levels=c("1","2","3","4","5"),
                     labels=c("Dalc_1","Dalc_2","Dalc_3","Dalc_4","Dalc_5"))
dataq[,"Walc"]<-factor(Data[,"Walc"],
                     levels=c("1","2","3","4","5"),
                     labels=c("Walc_1","Walc_2","Walc_3","Walc_4","Walc_5"))
dataq[,"health"]<-factor(Data[,"health"],
                     levels=c("1","2","3","4","5"),
                     labels=c("health_1","health_2","health_3","health_4","health_5"))

### Ajout de la variable 'Alc' qui prend en compte la somme de 'Walc' et 'Dalc' :
But : avoir un aperçu général de la consommation sur la semaine complète

In [ ]:
dataq[,"Alc"]<-factor(Data[,"Dalc"]+Data[,"Walc"])
levels(dataq[,"Alc"]) = c("1","1","2","2","3","3","4","4","5","5")
dataq[,"Alc"]<-factor(dataq[,"Alc"],
                     levels=c("1","2","3","4","5"),
                     labels=c("Alc_1","Alc_2","Alc_3","Alc_4","Alc_5"))

# 3 - Exploration des données

### Analyse des variables quantitatives

In [ ]:
corrplot(cor(Data[,variables_numeriques]), method="ellipse")

### Diagramme boîtes parallèles de chaque variable quantitative.

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
boxplot(Data[,variables_numeriques])

### Nuage de points des absences et des notes aux trois évaluations

In [ ]:
pairs(absences ~ G1 + G2 + G3, data=Data)

### Etude de la variable "Dalc" (consommation d'alcool en semaine)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
par(mfrow=c(1,2))
boxplot(Data[,"Dalc"]);hist(Data[,"Dalc"],breaks=6)

### Etude de la variable "Walc" (consommation d'alcool le weekend)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
par(mfrow=c(1,2))
boxplot(Data[,"Walc"]);hist(Data[,"Walc"])

### Etude combinée des variables "Dalc" et "Walc" (consommation globale d'alcool)

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
mosaicplot(~Dalc + Walc, data=Data)

### Lien entre résultats à l'examen final et consommation d'alcool

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)
par(mfrow=c(1,2))
boxplot(G3~Walc, data=Data, main="G3=f(Walc)");boxplot(G3~Dalc, data=Data, main="G3=f(Dalc)")

### Test d'égalité des moyennes

In [ ]:
note_walc1 = Data[Data$Walc==1,]$G3
note_walc2 = Data[Data$Walc==2,]$G3
note_walc3 = Data[Data$Walc==3,]$G3
note_walc4 = Data[Data$Walc==4,]$G3
note_walc5 = Data[Data$Walc==5,]$G3

In [ ]:
Notes = c(note_walc1,note_walc2,note_walc3,note_walc4,note_walc5)
conso_walc = c(rep("1",dim(as.matrix(note_walc1))[1]),rep("2",dim(as.matrix(note_walc2))[1]),
           rep("3",dim(as.matrix(note_walc3))[1]),rep("4",dim(as.matrix(note_walc4))[1]),rep("5",dim(as.matrix(note_walc5))[1]))


In [ ]:
an1 = lm(Notes~conso_walc)

In [ ]:
summary(an1)

In [ ]:
note_dalc1 = Data[Data$Dalc==1,]$G3
note_dalc2 = Data[Data$Dalc==2,]$G3
note_dalc3 = Data[Data$Dalc==3,]$G3
note_dalc4 = Data[Data$Dalc==4,]$G3
note_dalc5 = Data[Data$Dalc==5,]$G3

In [ ]:
Notes = c(note_dalc1,note_dalc2,note_dalc3,note_dalc4,note_dalc5)
conso_dalc = c(rep("1",dim(as.matrix(note_dalc1))[1]),rep("2",dim(as.matrix(note_dalc2))[1]),
           rep("3",dim(as.matrix(note_dalc3))[1]),rep("4",dim(as.matrix(note_dalc4))[1]),rep("5",dim(as.matrix(note_dalc5))[1]))


In [ ]:
an1 = lm(Notes~conso_dalc)

In [ ]:
summary(an1)

### Etude clinique entre Alcool et Failures

In [ ]:
table(Data$Walc,Data$failures)

In [ ]:
table(Data$Dalc,Data$failures)

In [ ]:
failures_bin = Data$failures
Dalc_regroupe = Data$Dalc
Walc_regroupe = Data$Walc

In [ ]:
for (i in 1:dim(Data["failures"])[1]){
    if(failures_bin[i]>1) failures_bin[i]=1
}
for (i in 1:dim(Data["Dalc"])[1]){
    if(Dalc_regroupe[i]>3) Dalc_regroupe[i]=3
}
for (i in 1:dim(Data["Walc"])[1]){
    if(Walc_regroupe[i]>4) Walc_regroupe[i]=4
}
#failures_bin
#Dalc_regroupe
#Walc_regroupe

On regroupe les modalités (échecs >= 1 et nb échecs =0 ) pour avoir des classes de plus forts effectifs.

In [ ]:
tab = table(Dalc_regroupe,failures_bin)
tab

In [ ]:
khi_test = chisq.test(tab)
khi_test

In [ ]:
tab2 = table(Walc_regroupe,failures_bin)
tab2

In [ ]:
khi_test2 = chisq.test(tab2)
khi_test2

### Lien entre la note finale et les variables qualitatives

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
par(mfrow=c(3,3))
boxplot(G3~school, data=Data,main="school");boxplot(G3~sex, data=Data,main="sex");boxplot(G3~address, data=Data,main="address")
boxplot(G3~famsize, data=Data,main="famsize");boxplot(G3~Pstatus, data=Data,main="Pstatus");boxplot(G3~Mjob, data=Data,main="Mjob")
boxplot(G3~Fjob, data=Data,main="Fjob");boxplot(G3~reason, data=Data,main="reason");boxplot(G3~guardian, data=Data,main="guardian")

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
par(mfrow=c(3,3))
boxplot(G3~schoolsup, data=Data,main="schoolsup");boxplot(G3~famsup, data=Data,main="famsup");boxplot(G3~paid, data=Data,main="paid")
boxplot(G3~activities, data=Data,main="activities");boxplot(G3~nursery, data=Data,main="nursery");boxplot(G3~higher, data=Data,main="higher")
boxplot(G3~internet, data=Data,main="internet");boxplot(G3~romantic, data=Data,main="romantic")

### Analyse de toutes les variables (y compris les quantitatives, transformées en qualitatives) : AFCM

In [ ]:
dim(dataq)
afc=MCA(dataq,quali.sup=c(31:34)) #,quali.sup=c(38)
summary(afc)

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)
plot.MCA(afc,invisible=c("ind"),col.var="blue",habillage="quali")

# 4 - Séparation des données en un échantillon "Apprentissage" et un "Test"

In [ ]:
set.seed(111)# initialisation de la suite aléatoire

npop=nrow(Data)
# tirage de 20% d'indices sans remise
n=0.2*npop
testi=sample(1:npop,n)

#Liste des indices restant qui n’ont pas été tirés
appri=setdiff(1:npop,testi)

# Extraction échantillons d’apprentissage
DataAppt=Data[appri,]
# Extraction échantillons de test
DataTest=Data[testi,]

In [ ]:
dim(Data)
dim(DataAppt)
dim(DataTest)

# 5 - Modèle linéaire

In [ ]:
plot.res=function(x,y,titre="titre")
{
plot(x,y,col="blue",xlim=c(0,20),ylim=c(-10,15),
ylab="Résidus",xlab="Valeurs predites",main=titre,pch=20)
abline(h=0,col="green")
}

## 5.1 - Modèle "Brut", avec toutes les variables

In [ ]:
# estimation du modèle sans interaction
reg.lm=glm(G3~.,data=DataAppt[,-c(31,32)])
# Extraction des résidus et des valeurs ajustées
# de ce modèle
res.lm=reg.lm$residuals
fit.lm=reg.lm$fitted.values
# graphe des résidus
# Définition d'une fonction pour un graphe coloré et 
# des échelles fixes sur les axes
options(repr.plot.width=10, repr.plot.height=10)
plot.res(fit.lm,res.lm,"Résidus en fonction des valeurs prédites")

In [ ]:
plot(fit.lm, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites",main="Valeurs observées en fonction des valeurs prédites pour la note G3")

In [ ]:
summary(reg.lm)

## 5.2 - Sélection de variables par algorithme Backward

In [ ]:
start_time <- Sys.time()

reg.lm_back=glm(G3~.,data=DataAppt[,-c(31,32)]) #On enlève G1 et G2 qui influencent trop G3
reg.lm_back.step=step(reg.lm_back,direction="backward")
end_time <- Sys.time()
t=end_time - start_time
t

In [ ]:
res.lm_back=reg.lm_back.step$residuals
fit.lm_back=reg.lm_back.step$fitted.values
plot.res(fit.lm_back,res.lm_back,"")

In [ ]:
plot(fit.lm_back, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites",main="Valeurs observées en fonction des valeurs prédites pour la note G3")

In [ ]:
summary(reg.lm_back.step)

In [ ]:
anova(reg.lm_back.step,test='F')

In [ ]:
cv.glm(DataAppt[,-c(31,32)],reg.lm_back.step,K=10)$delta[1]

## 5.3 - Sélection de variables par algorithme Forward

In [ ]:
start_time <- Sys.time()

reg.lm_for=glm(G3~1,data=DataAppt[,-c(31,32)])
reg.lm_for.step=step(reg.lm_for,direction="forward",scope=list(lower=~1,upper=~school+sex+address+famsize+Pstatus+Mjob+Fjob+reason+
                                                      guardian+schoolsup+famsup+paid+activities+nursery+higher+internet+
                                                      romantic+age+absences+Medu+Fedu+traveltime+studytime+failures+
                                                      famrel+freetime+goout+Dalc+Walc+health))
end_time <- Sys.time()
t=end_time - start_time
t

In [ ]:
res.lm_for=reg.lm_for.step$residuals
fit.lm_for=reg.lm_for.step$fitted.values
plot.res(fit.lm_for,res.lm_for,"")

In [ ]:
plot(fit.lm, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites",main="Valeurs observées en fonction des valeurs prédites pour la note G3")

In [ ]:
summary(reg.lm_for.step)

In [ ]:
anova(reg.lm_for.step,test='F')

In [ ]:
cv.glm(DataAppt[,-c(31,32)],reg.lm_for.step,K=10)$delta[1]

## 5.4 - Sélection de variables par algorithme Both

In [ ]:
start_time <- Sys.time()

reg.lm_both=glm(G3~1,data=DataAppt[,-c(31,32)])
reg.lm_both.step=step(reg.lm_both,direction="both",scope=list(lower=~1,upper=~school+sex+address+famsize+Pstatus+Mjob+Fjob+reason+
                                                      guardian+schoolsup+famsup+paid+activities+nursery+higher+internet+
                                                      romantic+age+absences+Medu+Fedu+traveltime+studytime+failures+
                                                      famrel+freetime+goout+Dalc+Walc+health))
end_time <- Sys.time()
t=end_time - start_time
t

In [ ]:
res.lm_both=reg.lm_both.step$residuals
fit.lm_both=reg.lm_both.step$fitted.values
plot.res(fit.lm_both,res.lm_both,"")

In [ ]:
plot(fit.lm_both, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites",main="Valeurs observées en fonction des valeurs prédites pour la note G3")

In [ ]:
summary(reg.lm_both.step)

In [ ]:
anova(reg.lm_both.step,test='F')

In [ ]:
cv.glm(DataAppt[,-c(31,32)],reg.lm_both.step,K=10)$delta[1]

# 6 - Modèle linéaire avec interactions

## 6.1 - Sélection de variables par algorithme Backward

Algorithme non implémenté : part d'un modèle trop compliqué et met par conséquent trop de temps à tourner.

In [ ]:
# Estimation du modèle de toute interaction d'ordre 2
    #reg.glm_back_quad=glm(G3~(.)^2,data=DataAppt[,-c(31,32)])
# Recherche du meilleur modèle au sens 
# du critère d'Akaïke par méthode descendante
    #reg.glm_back_quad.step=step(reg.glm_back_quad,direction="backward")

In [ ]:
# Coefficients du modèle
    #anova(reg.glm_back_quad.step,test="F")

In [ ]:
# Extraction des valeurs ajustées et des résidus
    #fit.glm_back_quad=reg.glm_back_quad.step$fitted.values
    #res.glm_back_quad=reg.glm_back_quad.step$residuals
# Graphe des résidus
    #plot.res(fit.glm_back_quad,res.glm_back_quad,"")

In [ ]:
    #cv.glm(DataAppt[,-c(31,32)],reg.glm_back_quad.step,K=10)$delta[1]

## 6.2 - Sélection de variables par algorithme Forward

In [ ]:
# Estimation du modèle de toute interaction d'ordre 2
start_time <- Sys.time()

reg.glm_for_quad=glm(G3~1,data=DataAppt[,-c(31,32)])
reg.glm_for_quad.step=step(reg.glm_for_quad,,direction="forward",scope=list(lower=~1,upper=~(school+sex+address+famsize+Pstatus+Mjob+Fjob+reason+
                                                      guardian+schoolsup+famsup+paid+activities+nursery+higher+internet+
                                                      romantic+age+absences+Medu+Fedu+traveltime+studytime+failures+
                                                      famrel+freetime+goout+Dalc+Walc+health)^2))
end_time <- Sys.time()
t=end_time - start_time
t

In [ ]:
# Coefficients du modèle
anova(reg.glm_for_quad.step,test="F")

In [ ]:
summary(reg.glm_for_quad.step)

In [ ]:
# Extraction des valeurs ajustées et des résidus
fit.glm_for_quad=reg.glm_for_quad.step$fitted.values
res.glm_for_quad=reg.glm_for_quad.step$residuals
# Graphe des résidus
plot.res(fit.glm_for_quad,res.glm_for_quad,"")

In [ ]:
plot(fit.glm_for_quad, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites",main="Valeurs observées en fonction des valeurs prédites pour la note G3")

In [ ]:
cv.glm(DataAppt[,-c(31,32)],reg.glm_for_quad.step,K=10)$delta[1]

## 6.3 - Sélection de variables par algorithme Both

In [ ]:
# Estimation du modèle de toute interaction d'ordre 2
start_time <- Sys.time()
reg.glm_both_quad=glm(G3~1,data=DataAppt[,-c(31,32)])
reg.glm_both_quad.step=step(reg.glm_both_quad,direction="both",scope=list(lower=~1,upper=~(school+sex+address+famsize+Pstatus+Mjob+Fjob+reason+
                                                      guardian+schoolsup+famsup+paid+activities+nursery+higher+internet+
                                                      romantic+age+absences+Medu+Fedu+traveltime+studytime+failures+
                                                      famrel+freetime+goout+Dalc+Walc+health)^2))
end_time <- Sys.time()
t=end_time - start_time
t

In [ ]:
# Coefficients du modèle
anova(reg.glm_both_quad.step,test="F")

In [ ]:
# Extraction des valeurs ajustées et des résidus
fit.glm_both_quad=reg.glm_both_quad.step$fitted.values
res.glm_both_quad=reg.glm_both_quad.step$residuals
# Graphe des résidus
plot.res(fit.glm_both_quad,res.glm_both_quad,"")

In [ ]:
plot(fit.glm_both_quad, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites",main="Valeurs observées en fonction des valeurs prédites pour la note G3")

In [ ]:
cv.glm(DataAppt[,-c(31,32)],reg.glm_both_quad.step,K=10)$delta[1]

# 7 - Bilan : comparaison entre les 5 modèles linéaires précédents

In [ ]:
"Modèle linéaire Backward :"
cv.glm(DataAppt[,-c(31,32)],reg.lm_back.step,K=10)$delta[1]
"Modèle linéaire Forward :"
cv.glm(DataAppt[,-c(31,32)],reg.lm_for.step,K=10)$delta[1]
"Modèle linéaire Both :"
cv.glm(DataAppt[,-c(31,32)],reg.lm_both.step,K=10)$delta[1]
"Modèle linéaire quadratique Forward :"
cv.glm(DataAppt[,-c(31,32)],reg.glm_for_quad.step,K=10)$delta[1]
"Modèle linéaire quadratique Both :"
cv.glm(DataAppt[,-c(31,32)],reg.glm_both_quad.step,K=10)$delta[1]

# 8 - Préivision sur l'échantillon test pour chacun des modèles précédents

#### Prévision modèle linéaire, par méthode backward

In [ ]:
predTest_back=predict(reg.lm_back.step,newdata=DataTest)
#predTest
table(ceiling(predTest_back),DataTest$G3)

In [ ]:
sum((predTest_back-DataTest[,"G3"])^2)/nrow(DataTest)

#### Prévision modèle linéaire, par méthode forward

In [ ]:
predTest_for=predict(reg.lm_for.step,newdata=DataTest)
#predTest
table(ceiling(predTest_for),DataTest$G3)

In [ ]:
sum((predTest_for-DataTest[,"G3"])^2)/nrow(DataTest)

#### Prévision modèle linéaire, par méthode both

In [ ]:
predTest_both=predict(reg.lm_both.step,newdata=DataTest)
#predTest
table(ceiling(predTest_both),DataTest$G3)

In [ ]:
sum((predTest_both-DataTest[,"G3"])^2)/nrow(DataTest)

#### Prévision modèle linéaire avec interaction, par méthode forward

In [ ]:
predTest_for_quad=predict(reg.glm_for_quad.step,newdata=DataTest)
#predTest
table(ceiling(predTest_for_quad),DataTest$G3)

In [ ]:
sum((predTest_for_quad-DataTest[,"G3"])^2)/nrow(DataTest)

#### Prévision modèle linéaire avec interaction, par méthode both

In [ ]:
predTest_both_quad=predict(reg.glm_both_quad.step,newdata=DataTest)
#predTest
table(ceiling(predTest_both_quad),DataTest$G3)

In [ ]:
sum((predTest_both_quad-DataTest[,"G3"])^2)/nrow(DataTest)

### Graphe valeurs observées en fonction des valeurs prédites

In [ ]:
par(mfrow=c(1,2))
plot(fit.lm_both, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites", main="Modèle linéaire");
plot(fit.glm_both_quad, DataAppt$G3, ylab="Valeurs observées",xlab="Valeurs predites", main="modèle quadratique")

# 9 - Arbre de décision

## 9.1 - Construction du modèle

In [ ]:
library(rpart)

In [ ]:
start_time <- Sys.time()

tree.reg=rpart(G3~.,data=DataAppt[,-c(31,32)],control=rpart.control(cp=0.001))

end_time <- Sys.time()
t=end_time - start_time
t

In [ ]:
plot(tree.reg)
text(tree.reg)

## 9.2 - Optimisation de l'élagage de l'arbre

In [ ]:
xmat=xpred.rpart(tree.reg)
xerr=(xmat-DataAppt[,-c(31,32)][,"G3"])^2
CVerr=apply(xerr,2,sum)
CVerr  #    CP           erreur

In [ ]:
as.numeric(attributes(which.min(CVerr))$names) #Le cp qui minimise l'erreur pour faire un arbre simplifié

In [ ]:
start_time <- Sys.time()

tree.reg=rpart(G3~.,data=DataAppt[,-c(31,32)],control=rpart.control(cp=as.numeric(attributes(which.min(CVerr))$names)))

end_time <- Sys.time()
t=end_time - start_time
t


In [ ]:
library(partykit)

In [ ]:
options(repr.plot.width=13, repr.plot.height=8)

plot(as.party(tree.reg), type="simple")

## 9.3 - Prédiction sur l'échantillon test

In [ ]:
#Prédiction sur l'échantillon test
predTest_DT=predict(tree.reg,newdata=DataTest)


# Arrondi des prédiction pour comparaison avec valeurs réelles
n = dim(DataTest)[1]

predTest_rounded = predTest_DT
for (i in 1:n){
    if (ceiling(predTest_rounded[i])-predTest_rounded[i] <0.5){
        predTest_rounded[i] = ceiling(predTest_rounded[i])
    } else{
        predTest_rounded[i] = floor(predTest_rounded[i])
    }
}

#affichage
table(ceiling(predTest_rounded),DataTest$G3)
sum((predTest_rounded-DataTest[,"G3"])^2)/nrow(DataTest)

# 10 - Random Forest

## 10.1 - Construction du modèle

In [ ]:
library(randomForest)

In [ ]:
start_time <- Sys.time()

rf.reg=randomForest(G3~., data=DataAppt[,-c(31,32)],xtest=DataTest[,-c(31,32,33)],ytest=DataTest[,"G3"],
   ntree=500,do.trace=50,importance=TRUE, mtry=10)

end_time <- Sys.time()
t=end_time - start_time
t


In [ ]:
fit.rfr=rf.reg$predicted
res.rfr=fit.rfr-DataAppt[,"G3"]
plot.res(fit.rfr,res.rfr,titre="")

## 10.2 - Prévision sur l'échantillon test

In [ ]:
pred.rf=rf.reg$test$predicted

table(ceiling(pred.rf),DataTest$G3)
# Erreur quadratique moyenne de prévision
sum((pred.rf-DataTest[,"G3"])^2)/nrow(DataTest)

## 10.3 - Importance des variables pour le Random Forest

In [ ]:
sort(round(importance(rf.reg), 2)[,1], decreasing=TRUE)

## 10.4 - Avec le package Ranger (au lieu de Random Forest)

In [ ]:
library(ranger)

In [ ]:
start_time <- Sys.time()
rf <- ranger(G3 ~ ., data=DataAppt[,-c(31,32)], num.trees = 500,
             importance ='impurity_corrected', mtry = 10)#, min.node.size=3)

end_time <- Sys.time()
t=end_time - start_time
t

pred <- predict(rf, DataTest) 
#pred$predictions

table(ceiling(pred$predictions),DataTest$G3)
# Erreur quadratique moyenne de prévision
sum((pred$predictions-DataTest[,"G3"])^2)/nrow(DataTest)

In [ ]:
fit.rfr = predictions(rf, num.trees = ranger_RF$num.trees)
res.rfr=fit.rfr-DataAppt[,"G3"]
plot.res(fit.rfr,res.rfr,titre="")

In [ ]:
sort(rf$variable.importance, decreasing=TRUE)

# 11 - Algorithme de Boosting : GBM

## 11.1 - Construction du modèle

In [ ]:
library(gbm)

In [ ]:
n_trees=80

In [ ]:
boost.reg=gbm(G3~., data=DataAppt,distribution="gaussian",n.trees=n_trees, cv.folds=10,
        n.minobsinnode = 5,shrinkage=0.1,verbose=FALSE)
plot(boost.reg$cv.error)

In [ ]:
# nombre optimal d'itérations par valiation croisée
best.iter=gbm.perf(boost.reg,method="cv")

In [ ]:
test=numeric()
for (i in 10:n_trees){
pred.test=predict(boost.reg,newdata=DataTest,n.trees=i)
err=sum((pred.test-DataTest[,"G3"])^2)/nrow(DataTest)
test=c(test,err)}
plot(10:n_trees,test,type="l")
abline(v=best.iter)

In [ ]:
fit.boostr=boost.reg$fit
res.boostr=fit.boostr-DataAppt[,"G3"]
plot.res(fit.boostr,res.boostr,titre="")

## 11.2 - Prédiction sur l'échantillon test

In [ ]:
best.iter

pred <- predict(boost.reg, DataTest,n.trees=best.iter) 

pred

In [ ]:
table(ceiling(pred),DataTest$G3)
# Erreur quadratique moyenne de prévision
sum((pred-DataTest[,"G3"])^2)/nrow(DataTest)

# 12 - Utilisation du package Caret

Réalisation de 50 itérations par méthode pour évaluer le temps de calcul moyen d'exécution des algorithmes.

In [ ]:
library(caret)


## 12.1 - Séparation Apprentissage / Test

In [ ]:
Y=Data[,"G3"]
X=Data[,-c(31,32,33)]

In [ ]:
xx=11
set.seed(xx)
inTrain = createDataPartition(X[,1],p = 0.8, list = FALSE)
# Extraction des échantillons
trainDescr=X[inTrain,]
testDescr=X[-inTrain,]
testY=Y[-inTrain]
trainY=Y[inTrain]

In [ ]:
# Normalisation calculée sur les paramètres de l'échantillon d'apprentissage
xTrans=preProcess(trainDescr)
trainDescr=predict(xTrans,trainDescr)
# Puis appliquée également à l'échantillon test
testDescr=predict(xTrans,testDescr)
# Choix de la validation croisée
cvControl=trainControl(method="cv",number=10)

## 12.2 - Modèle linéaire

In [ ]:
#Régression logistique
# Attention, la régression logistique sans interaction (linéaire) est estimée ci-dessous
t_tot = vector("numeric", 50)
for (i in 1:50){
    start_time <- Sys.time()
    set.seed(2)
    rlogFit = train(trainDescr, trainY,method = "glmStepAIC", tuneLength = 10,
                    trControl = cvControl)

    end_time <- Sys.time()
    t=end_time - start_time
    t_tot[i]=t
    }

In [ ]:
t_tot
mean(t_tot)

## 12.3 - Arbre de décision

In [ ]:
#Arbre de décision
t_tot= vector("numeric", 50)

for (i in 1:50) {
    start_time <- Sys.time()

    set.seed(2)
    rpartFit = train(trainDescr, trainY, method = "rpart", tuneLength = 10,
        trControl = cvControl)

    end_time <- Sys.time()
    t=end_time - start_time
    t_tot[i]=t

    #rpartFit
    #plot(rpartFit)
    }
t_tot
mean(t_tot)

In [ ]:
summary(rpartFit)

## 12.4 - Random Forest

In [ ]:
#Random forest
t_tot= vector("numeric", 50)
for (i in 1:50) {
    start_time <- Sys.time()

    set.seed(2)
    rfFit = train(trainDescr, trainY,method = "rf", tuneLength = 8,
                  trControl = cvControl, trace=FALSE)

    end_time <- Sys.time()
    t=end_time - start_time
    t_tot[i]=t

    #rfFit
    #plot(rfFit)
    }
t_tot
mean(t_tot)

In [ ]:
rfFit$finalModel

In [ ]:
summary(rfFit)

## 12.5 - GBM

In [ ]:
# Boosting 
t_tot= vector("numeric", 50)
for (i in 1:50) {
    start_time <- Sys.time()
    
    set.seed(2)
    gbmFit = train(trainDescr, trainY,method = "gbm", tuneLength = 8,
                   trControl = cvControl)
    
    end_time <- Sys.time()
    t=end_time - start_time
    t_tot[i]=t
    
    #gbmFit
    #plot(gbmFit)
    }
t_tot
mean(t_tot)

In [ ]:
summary(gbmFit)

## 12.6 - XGB

In [ ]:
# Extrême boosting
t_tot= vector("numeric", 50)
for (i in 1:50) {
    start_time <- Sys.time()
    set.seed(2)
    xgbFit = train(trainDescr, trainY,method = "xgbTree", tuneLength = 6,
               trControl = cvControl, trace=FALSE)
    #xgbFit
    #plot(xgbFit)
    end_time <- Sys.time()
    t=end_time - start_time
    t_tot[i]=t
    }
t_tot
mean(t_tot)

## 12.7 - Prédiction de chacun des modèles précédent

In [ ]:
models=list(logit=rlogFit,cart=rpartFit,rf=rfFit,gbm=gbmFit)#,xgb=xgbFit)
testPred=predict(models, newdata = testDescr)
#testPred

In [ ]:
# Erreur quadratique moyenne de prévision
"Régression linéaire :"
sum((testPred$logit-testY)^2)/nrow(testDescr)

"Arbre de décision :"
sum((testPred$cart-testY)^2)/nrow(testDescr)

"Random Forest :"
sum((testPred$rf-testY)^2)/nrow(testDescr)

"Boosting (GBM) : "
sum((testPred$gbm-testY)^2)/nrow(testDescr)

#"Extreme GBM :"
#sum((testPred$xgb-testY)^2)/nrow(DataTest)

# 13 - Itération par validation croisée de Monte Carlo

## 13.1 - Réalisation de la validation croisée sur 50 itérations

In [ ]:
pred.autom=function(X,Y,p=1/2,methodes=c("knn",
"rf"),size=c(10,2),xinit=11,N=10,typerr="cv",
number=4,type="raw") {
# Fonction de prévision de N échantillons tests
# par une liste de méthodes de régression
# ou classification (uniquement 2 classes)
# Optimisation des paramètres par validation
# croisée (défaut) ou bootstrap ou... (cf. caret)
# X : matrice ou frame des variables explicatives
# Y : variable cible quantitative ou qualitative
# p : proportion entre apprentissage et test
# methodes : liste des méthodes de rdiscrimination
# size : e grille des paramètres à optimiser
# xinit : générateur de nombres aléatoires
# N : nombre de réplications apprentissage/test
# typerr : "cv" ou "boo" ou "oob"
# number : nombre de répétitions CV ou bootstrap
# pred : liste des matrices de prévision
# type d’erreur
Control=trainControl(method=typerr,number=number)
# initialisation du générateur
set.seed(xinit)
# liste de matrices stockant les prévisions
# une par méthode
inTrain=createDataPartition(Y,p=p,list=FALSE)
ntest=length(Y[-inTrain])
pred=vector("list",length(methodes))
names(pred)=methodes
pred=lapply(pred,function(x)x=matrix(0,
nrow=ntest,ncol=N))
obs=matrix(0,ntest,N)
set.seed(xinit)
for(i in 1:N) {
# N itérations
# indices de l’échantillon d’apprentissage
inTrain=createDataPartition(Y,p=p,list=FALSE)
# Extraction des échantillons
trainDescr=X[inTrain,]
testDescr=X[-inTrain,]
trainY=Y[inTrain]
testY=Y[-inTrain]
# stockage des observés de testY
obs[,i]=testY
# centrage et réduction des variables
xTrans=preProcess(trainDescr)
trainDescr=predict(xTrans,trainDescr)
testDescr=predict(xTrans,testDescr)
# estimation et optimisation des modèles
# pour chaque méthode de la liste
for(j in 1:length(methodes)) {
# modélisation
modFit = train(trainDescr, trainY,method = methodes[j], tuneLength = size[j],
               trControl = Control)
print(summary(modFit))
# prévisions
if (type=="prob")  pred[[j]][,i]=predict(modFit,
newdata = testDescr,type=type)[,1]
else pred[[j]][,i]=predict(modFit,
newdata = testDescr)
}}
list(pred=pred,obs=obs)
# résultats
}

In [ ]:
X = Data[,-c(31,32,33)]
Y = Data[,33]


In [ ]:
# Choisir la liste des méthodes et l’effort d’optimisation
models=c("gbm","glmStepAIC","rpart","rf")#,"xgb"
noptim=c(6,6,6,6)#,6)

Niter=50 ; Init=11 
# Appel de la fonction définie ci-dessus
pred.g3=pred.autom(X,Y,p=0.8,methodes=models,N=Niter,xinit=Init,size=noptim)

In [ ]:
# Calcul des taux de bien classés
obs=pred.g3$obs
prev.g3=pred.g3$pred

res_GLM.g3 = vector("numeric", Niter)
res_RF.g3 = vector("numeric", Niter)
res_DT.g3 = vector("numeric", Niter)
res_GBM.g3 = vector("numeric", Niter)
#res_XGB.g3 = vector("numeric", Niter)

for (i in 1:Niter){
    res_GLM.g3[i]=sum((prev.g3$glmStepAIC[,i]-obs[,i])^2)/nrow(DataTest)
    res_RF.g3[i]=sum((prev.g3$rf[,i]-obs[,i])^2)/nrow(DataTest)
    res_DT.g3[i]=sum((prev.g3$rpart[,i]-obs[,i])^2)/nrow(DataTest)
    res_GBM.g3[i]=sum((prev.g3$gbm[,i]-obs[,i])^2)/nrow(DataTest)
    #res_XGB.g3[i]=sum((prev.g3$xgb[,i]-obs[,i])^2)/nrow(DataTest)
}

# distributions des erreurs 
boxplot(res_GLM.g3,res_RF.g3,res_DT.g3,res_GBM.g3,xlab="GLM                  -                RF                    -                     DT                   -                    GBM")

## 13.2 - Comparaison des moyennes des erreurs des méthodes

In [ ]:
Erreurs = c(res_GLM.g3,res_RF.g3,res_DT.g3,res_GBM.g3)
Method = c(rep("GLM",Niter),rep("RF",Niter),rep("DT",Niter),rep("GBM",Niter))
Data_err = data.frame(Erreurs=Erreurs, Method=Method)

In [ ]:
an1 = lm(Erreurs~Method)

In [ ]:
summary(an1)

On enlève l'arbre qui a une erreur de prédiction moyenne nettement différente des autres modèles

In [ ]:
Erreurs2 = c(res_GLM.g3,res_RF.g3,res_GBM.g3)
Method2 = c(rep("GLM",Niter),rep("RF",Niter),rep("GBM",Niter))
Data_err2 = data.frame(Erreurs2=Erreurs2, Method2=Method2)

an2 = lm(Erreurs2~Method2)

summary(an2)